# Virtual Machines (VM)


  > A hypervisor or virtual machine monitor (VMM) is computer software, firmware or hardware that creates and runs virtual machines. A computer on which a hypervisor runs one or more virtual machines is called a host machine, and each virtual machine is called a guest machine. The hypervisor presents the guest operating systems with a virtual operating platform and manages the execution of the guest operating systems. Multiple instances of a variety of operating systems may share the virtualized hardware resources: for example, Linux, Windows, and macOS instances can all run on a single physical x86 machine. This contrasts with operating-system-level virtualization, where all instances (usually called containers) must share a single kernel, though the guest operating systems can differ in user space, such as different Linux distributions with the same kernel. (https://en.wikipedia.org/wiki/Hypervisor)

![](https://upload.wikimedia.org/wikipedia/commons/e/e1/Hyperviseur.png)

  > Virtual machines run guest operating systems—note the OS layer in each box. This is resource intensive, and the resulting disk image and application state is an entanglement of OS settings, system-installed dependencies, OS security patches, and other easy-to-lose, hard-to-replicate ephemera. (https://docs.docker.com/get-started/#containers-vs-virtual-machines)
  
![](images/VM.png)









There are many different tools for running VMs. Some of them are:

  * [Parallels Workstation](http://www.parallels.com/eu/all-products)
  * [VirtualBox](http://virtualbox.org)
  * [VMware](https://www.vmware.com) 
  * [QEMU](https://www.qemu.org)
  


# Containers

![](images/Container.png)

  > Containers can share a single kernel, and the only information that needs to be in a container image is the executable and its package dependencies, which never need to be installed on the host system. These processes run like native processes, and you can manage them individually by running commands like `docker ps`—just like you would run `ps` on Linux to see active processes. Finally, because they contain all their dependencies, there is no configuration entanglement; a containerized app "runs anywhere." (https://docs.docker.com/get-started/#container-diagram)
  
  
Again, there are many different tools (hypervisors) for running containers. Some of them are:

  * [Docker](https://www.docker.com)
  * [VServer](http://linux-vserver.org/Welcome_to_Linux-VServer.org)
  * [LXC](https://linuxcontainers.org)
  * [rkt](https://coreos.com/rkt/)

Since it seems that Docker is the most popular and adoptet at the moment, we are going to have a look at Docker.




----------------------------------------


# Hands-on VMs with Vagrant

  > Vagrant is a tool for building and managing virtual machine environments in a single workflow. With an easy-to-use workflow and focus on automation, Vagrant lowers development environment setup time, increases production parity, and makes the "works on my machine" excuse a relic of the past.

  > Vagrant provides easy to configure, reproducible, and portable work environments built on top of industry-standard technology and controlled by a single consistent workflow to help maximize the productivity and flexibility of you and your team. (https://www.vagrantup.com)


## Intro to Vagrant

### Initialize & Start a VM

You can start your setup by creating a `Vagrantfile`. The argument to `vagrant init` is the name of the *box*. That is, 

```bash
$ mkdir vm
$ cd vm
$ vagrant init bento/ubuntu-16.04
$ vagrant up
```

  > After running the above commands, you will have a fully functional VM in VirtualBox running Ubuntu 16.04 LTS 64-bit. You can SSH into this machine with vagrant ssh, and when you are done playing around, you can terminate the virtual machine with vagrant destroy.
  
You can find a catalogue of available boxes at https://app.vagrantup.com/boxes/search

That is, you can quickly setup development environments for many operating systems. For example, in the following the initialization and use of a FreeBSD and Windows VM.

```bash
$ mkdir vm_freebsd
$ cd vm_freebsd
$ vagrant init freebsd/FreeBSD-11.0-STABLE
$ vagrant up
```

```bash
$ mkdir vm_win
$ cd vm
$ vagrant init opentable/win-2012r2-standard-amd64-nocm
$ vagrant up
```

**OBS:** From now on there are no excuses anymore like: *"I have a Windows laptop and cannot do that Linux exercise..."*!


### Accessing a VM

You log onto a VM with SSH as in the following.

```bash
$ vagrant ssh
```

You can leave the guest VM with:

```bash
vagrant@vagrant $ exit
$ 
```


### Teardown

You can put a running VM 

  * to sleep 
  
  ```bash
  vagrant suspend
  ```
  
  * turn it off 
  
  ```bash
  vagrant halt
  ``` 
  * or remove it completely 
  
  ```bash
  vagrant destroy
  ```



### Different Providers

In this tutorial, we always used VirtualBox as a provider. However, you can use many other, such as _VMWare_. But more importanlty, you can provision your VMs also to remote servers in the cloud.

```bash
$ vagrant up --provider=digital_ocean
$ vagrant up --provider=aws
```

The first line would deploy your VM as droplet on DigitalOcean (https://github.com/devopsgroup-io/vagrant-digitalocean) and the second line would do similarly on Amazon AWS (https://github.com/mitchellh/vagrant-aws).


### A Closer Look to a Vagrantfile

A `Vagrantfile` is just a Ruby script. That is, you write code to specify your development infrastructure.
A provision script can be a plain shell script or one of many other solutions, such as [Ansible](https://docs.ansible.com/), [Puppet](https://www.puppetlabs.com/puppet), or [Chef](https://www.chef.io/chef/).

However, we will use only Bash scripts for provisioning in the lecture. In your project you can apply whatever suits you best.

```ruby
# -*- mode: ruby -*-
# vi: set ft=ruby :

Vagrant.configure("2") do |config|
  config.vm.box = "bento/ubuntu-16.04"
  config.vm.network "forwarded_port", guest: 80, host: 8080
  config.vm.network "forwarded_port", guest: 8080, host: 8081
  config.vm.network "forwarded_port", guest: 8888, host: 8888
  config.vm.synced_folder "../", "/synced_folder"

  config.vm.provider "virtualbox" do |vb|
    vb.name = "lsd2017vm"
    vb.memory = "1024"
    vb.cpus = "2"
    # You might want to include the following for a GUI
    # vb.gui = true
  end

  config.vm.provision "file", source: "./data.csv", destination: "data.csv"

  config.vm.provision "shell", privileged: false, inline: <<-SHELL
    sudo apt-get update
    sudo apt-get install -y git
    sudo apt-get install -y wget

    echo "Installing Java..."
    sudo apt-get -y install software-properties-common
    sudo add-apt-repository ppa:webupd8team/java
    sudo apt-get -y update
    echo oracle-java8-installer shared/accepted-oracle-license-v1-1 select true | sudo /usr/bin/debconf-set-selections
    sudo apt-get -y install oracle-java8-installer
    sudo update-java-alternatives -s java-8-oracle

    echo "Installing Go..."
    wget https://storage.googleapis.com/golang/go1.8.3.linux-amd64.tar.gz
    sudo tar -C /usr/local -xzf go1.8.3.linux-amd64.tar.gz
    echo "export PATH=$PATH:/usr/local/go/bin" >> $HOME/.profile
    echo "export GOPATH=/go_projects" >> $HOME/.profile

    echo "Installing Anaconda..."
    sudo wget https://repo.continuum.io/archive/Anaconda3-4.4.0-Linux-x86_64.sh -O ~/Anaconda3-4.4.0-Linux-x86_64.sh
    bash ~/Anaconda3-4.4.0-Linux-x86_64.sh -b
    echo ". $HOME/.bashrc" >> $HOME/.bash_profile
    echo "export PATH=$HOME/anaconda3/bin:$PATH" >> $HOME/.bash_profile
    echo "alias notebook='jupyter notebook --no-browser --ip=0.0.0.0 --NotebookApp.token=\\"\\"'" >> $HOME/.bash_profile
    export PATH="$HOME/anaconda3/bin:$PATH"
    $(which pip) install suplemon

    # this package is necessary for matplotlib to work and not installed by 
    # default
    sudo apt-get install -y libgl1-mesa-glx

    echo "==================================================================="
    echo "=                             DONE                                ="
    echo "==================================================================="
    echo "To log onto the VM:"
    echo "$ vagrant ssh"
    echo "To switch to the directory with teaching material:"
    echo "$ cd /synced_folder/notebooks"
    echo "To start the Jupyter notebook server:"
    echo "$ notebook"
  SHELL
end
```


#### More than one VM per `Vagrantfile`

You can specify a cluster of VMs for your project. For example as in the following

```ruby
# -*- mode: ruby -*-
# vi: set ft=ruby :

Vagrant.configure("2") do |config|
  config.vm.box = "bento/ubuntu-16.04"

  config.vm.network "private_network", type: "dhcp"

  config.vm.define "webserver1", primary: true do |server|
    server.vm.network "private_network", ip: "192.168.20.2"
    server.vm.network "forwarded_port", guest: 8080, host: 8080
    server.vm.provider "virtualbox" do |vb|
      vb.memory = "1024"
      vb.cpus = "1"
    end
    server.vm.hostname = "webserver1"
    server.vm.provision "shell", inline: <<-SHELL
      echo "Hej from server one!" > /var/www/html/index.html
    SHELL
  end

  config.vm.define "webserver2" do |client|
    client.vm.network "private_network", ip: "192.168.20.3"
    client.vm.network "forwarded_port", guest: 8080, host: 9080
    client.vm.provider "virtualbox" do |vb|
      vb.memory = "1024"
      vb.cpus = "1"
    end
    client.vm.hostname = "webserver2"
    client.vm.provision "shell", inline: <<-SHELL
      echo "Hej from server 2!" > /var/www/html/index.html
    SHELL
  end

  config.vm.provision "shell", privileged: false, inline: <<-SHELL
    sudo apt-get update
    sudo apt-get -y install apache2
  SHELL
end
```

# Your Turn!

![](https://camo.githubusercontent.com/320b4791da998fd94e34ad4a85d44d8d5a581ca4/68747470733a2f2f732d6d656469612d63616368652d616b302e70696e696d672e636f6d2f6f726967696e616c732f39662f37332f65332f39663733653366386139353864626530336230663736663838313161353461312e676966)

Create a small application consisting of two components. A webserver written in your favorite programming language and a database server written in your favorite programming language. It does not really matter what is stored in the database and what is served by the webserver. However, both components have to communicate with each other automatically and a human user communicates with the webserver via a webclient.

That is, in case you would build a small telephone book application similar to Krak.dk, you would have subsystems communicating with each other as depicted in the following UML sequence diagram.

![](images/vm_sequence_diagram.png)

Your actual task is to create a `Vagrantfile` to setup two VMs. One for the webserver and another one for the database server. That is, your application is developed and deployed as illustrated in the following UML deployment diagram.

![](images/vm_deployment_small.png)

Hints: 

  * Use `bento/ubuntu-16.04` as the base box of the two VMs.
  * Start by building empty VMs.
  * Assign each VM a static IP.
  * The [`Vagrantfile`](https://github.com/HelgeCPH/db_course_nosql/blob/master/vm/Vagrantfile) of last semester's database course shows you how to setup different databases.
  * In case you need an example webserver and database server, Helge could provide you those.

----------------------------------------


# Hands-on, Containers with Docker


  > A container image is a lightweight, stand-alone, executable package of a piece of software that includes everything needed to run it: code, runtime, system tools, system libraries, settings.
  > 
  > ...
  > 
  > Containers isolate software from its surroundings, for example differences between development and staging environments and help reduce conflicts between teams running different software on the same infrastructure. (https://www.docker.com/what-container)
  
In case that does not really help then perhaps Liz Rice's code examples are more clear: https://www.youtube.com/watch?v=HPuvDm8IC-4  


  
## Hov, I think I know already what you want to tell us about Docker!

That is cool! Then please create new expertise and support your fellow students. 

  * Read the Docker documentation on networking, especially the sections   
    - https://docs.docker.com/engine/userguide/networking/ and 
    - https://docs.docker.com/engine/userguide/networking/work-with-networks/#create-networks
  * Create a markdown file in a repository on Github, in which you explain how to enable Docker container's communicating via a network. 
  
The reason for this task is, that the mechanism of enabling network communication with help of `--link`ing containers in the bridge network is now _deprecated_.


## Running my first container!

```bash
$ docker run --rm hello-world
Unable to find image 'hello-world:latest' locally
latest: Pulling from library/hello-world
b04784fba78d: Pull complete
Digest: sha256:f3b3b28a45160805bb16542c9531888519430e9e6d6ffc09d72261b0d26ff74f
Status: Downloaded newer image for hello-world:latest

Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash
```

The command above downloaded the image `hello-world` from the [Docker Hub](https://hub.docker.com), instantiated a container from that image, ran the application within this container, and finally deleted the container (`--rm`).

### I am on Windows and need a Linux Shell quickly!

Check on [Docker Hub](https://hub.docker.com), there are images for many different flavors of Linux and for many packaged applications.

```bash
$ docker run -it --rm alpine:latest sh
```

What does that do? It tells Docker to run a container with the latest version of Alpine Linux (a small Linux Distribution), connect to the shell process `sh`, run it interactively `-it` so that you can type in commands and see the results, and finally, to remove the container `--rm` after exiting from the container.


### Volumes

You can mount directories (_volumes_) from your host to a container using the `-v` flag.

```bash
$ docker run -it -v $(pwd):/host alpine:latest /bin/sh
```



### Development with Containers

Let's build a simple webserver in [Go](https://golang.org). To not mess with our development machine we could use a Docker container, which has the Go compiler readily installed.

Save the following file as `basic_http_server.go` in a directory `./webserver`.

```go
package main

import (
	"fmt"
	"log"
	"net/http"
)

func hejVerdenHandler(w http.ResponseWriter, r *http.Request) {
	fmt.Fprint(w, "Hej verden!\n")
}

func main() {
	port := 8080

	http.HandleFunc("/", hejVerdenHandler)

	log.Printf("Server starting on port %v\n", port)
	http.ListenAndServe(fmt.Sprintf(":%v", port), nil)
}
```

To dockerize that program, or better to run that program in a container without installing the compiler to our machines directly, you could run:

```bash
docker run -it --rm -v $(pwd)/webserver:/src -p 8080:8080 -w /src golang:jessie go run basic_http_server.go
```

That command instantiates a container of the image `golang:jessie` (an Ubuntu Linux with Go and some other necessary tools readily installed). Furthermore, we share our local code in `./webserver` with the container. There it is mounted to the `/src` directory. With `-w` we change the current working directory in that container to `/src`. Additionally, `-p 8080:8080` tells Docker to forward the port `8080` from the container to the same port number on our host. The port number in front of `:` specifies the port on the host, which gets bound to which port of the container (number after `:`). Finally, we run the program within the container `go run basic_http_server.go`.

Note, you could also build the program in the container and run the resulting binary.

```bash
docker run -it --rm -v $(pwd)/webserver:/src -p 8080:8080 -w /src golang:jessie go build basic_http_server.go; ./basic_http_server
```

Now, you can access the webserver on your host machine on http://127.0.0.1:8080. If you point your browser to that address you should see the following:

![](images/simple_dockerized_webapp.png)

Alternatively, you could run `curl` on your host machine to see that our server is working correctly. 

```bash
$ curl -s http://127.0.0.1:8080
```


Unfortunately, many operating systems do not come with the `curl` program installed. But likely there is a dockerized version of this program. If you run:

```bash
$ docker run --rm --link myserver appropriate/curl:latest curl -s http://myserver:8080
Hej verden!
```

then you downlad an image with a small Linux and with `curl` installed. However, the command above also allows the `curl` client to see our webserver `--link myserver`. Try to run the command above without that flag and see what happens.

To find more Docker images and dockerized programs have a look at https://hub.docker.com.

## `Dockerfile`s

`Dockerfile`s are similar to the `Vagrantfile`s that we discussed earlier. They describe exactly the configuration of a container. Unlike `Vagrantfile`s these configurations are stored as slices on top of each other.

Let's have a look on an example application. It will consist of a webserver and of a simple client. The webserver serves a static HTTP message on port 8080 and the client is just an HTTP `GET` query receiving this message via `curl`. The following UML deployment diagram illustrates this setup

![](images/vm_deployment_basic.png)


Let's have a look at a `Dockerfile` that specifies our webserver.

```Dockerfile
FROM golang:jessie

# Install any needed dependencies...
# RUN go get ...

# Set the working directory
WORKDIR /src

# Copy the server code into the container
COPY basic_http_server.go /src/basic_http_server.go

# Make port 8080 available to the host
EXPOSE 8080

# Build and run the server when the container is started
RUN go build /src/basic_http_server.go
ENTRYPOINT ./basic_http_server
```

As you can see from the above configuration, the `Dockerfile` is similar to everything described in our earlier CLI command:

```bash
$ docker run -it --rm -v $(pwd)/webserver:/src -p 8080:8080 -w /src golang:jessie go run basic_http_server.go
```

Keywords in `Dockerfile`s are `FROM`, `MAINTAINER`, `LABEL`, `RUN`, `CMD`, `EXPOSE`, `ENV`, ADD or `COPY`, `ENTRYPOINT`, `VOLUME`, `USER`, `WORKDIR`, `ONBUILD`. You can read more on them in the documentation: https://docs.docker.com/engine/userguide/eng-image/dockerfile_best-practices/#the-dockerfile-instructions.


### Building the Webserver Image

To use containers with our webserver, we first have to build a corresponding image. If you have the above `Dockerfile` stored in a directory `webserver` you can do so as in the following:

```bash
$ cd webserver/
$ docker build -t your_id/myserver .
```

The `-t` flag tells Docker to build an image with the given name `your_id/myserver`. The `.` says: _build the image with the `Dockerfile` in this directory_.

After building your image, you can verify that it is now accessible on your machine.

```bash
$ docker images
REPOSITORY           TAG                 IMAGE ID            CREATED             SIZE
your_id/myserver     latest              a5fe35de13d2        8 seconds ago       704MB
appropriate/curl     latest              f73fee23ac74        3 weeks ago         5.35MB
golang               jessie              6ce094895555        4 weeks ago         699MB
```


### Running the Webserver as Container

```bash
$ docker run --name webserver -p 8080:8080 your_id/myserver
```

### Stopping and Restarting the Webserver

```bash
$ docker stop webserver
```

```bash
$ docker start webserver
```


### Building the Client Image


```Dockerfile
FROM appropriate/curl:latest

ENTRYPOINT curl -s http://myserver:8080
```

### Building the Client Image

```bash
cd client/
docker build -t your_id/myclient .
```

```bash
$ docker images
REPOSITORY           TAG                 IMAGE ID            CREATED             SIZE
your_id/myclient     latest              3714e67fa75a        4 seconds ago       5.35MB
your_id/myserver     latest              a5fe35de13d2        About an hour ago   704MB
appropriate/curl     latest              f73fee23ac74        3 weeks ago         5.35MB
golang               jessie              6ce094895555        4 weeks ago         699MB
```


### Running the Webserver as Container

```bash
$ docker run --name client --link webserver your_id/myclient
Hej verden!
```

That is nice, is not it? We just build a small application consisting of a webserver and a client, both deployed in their own containers, and we did not have to install any dependencies on our host machine manually.

However, starting the server and the client by hand with the `docker run ...` command is quite tedious. Furthermore, it is not really in line with the _infrastructure as code_ paradigm. Therefore, we can automate even further using `docker-compose`.


## `docker-compose` - Starting the Application Automatically

Similar to a `Vagrantfile`, which describes a cluster setup, we can use a `docker-compose.yml` file to specify the components of our application and how they shall be started.


```yml
version: '3'
services:
  webserver:
    image: your_id/myserver
    ports:
      - "8080:8080"

  clidownload:
    image: appropriate/curl
    links:
      - myserver
    entrypoint: sh -c  "sleep 5 && curl -s http://webserver:8080"
```

```bash
$ docker-compose up
Creating 03containersandvms_webserver_1 ...
Creating 03containersandvms_webserver_1 ... done
Creating 03containersandvms_clidownload_1 ...
Creating 03containersandvms_clidownload_1 ... done
Attaching to 03containersandvms_webserver_1, 03containersandvms_clidownload_1
webserver_1    | 2017/08/23 16:02:17 Server starting on port 8080
clidownload_1  | Hej verden!
03containersandvms_clidownload_1 exited with code 0
^CGracefully stopping... (press Ctrl+C again to force)
Stopping 03containersandvms_webserver_1 ... done
```


```bash
$ docker-compose rm -v
```




# Your Turn!

![](https://camo.githubusercontent.com/320b4791da998fd94e34ad4a85d44d8d5a581ca4/68747470733a2f2f732d6d656469612d63616368652d616b302e70696e696d672e636f6d2f6f726967696e616c732f39662f37332f65332f39663733653366386139353864626530336230663736663838313161353461312e676966)

Similar as above in the Vagrant setup, you create a small application. Now however, it consists of *three* components. A webserver written in your favorite programming language, a database server written in your favorite programming language, and the webclient `curl` deployed in its own container. It does not really matter what is stored in the database and what is served by the webserver.

That is, in case you would build a small telephone book application similar to Krak.dk, you would have subsystems communicating with each other as depicted in the following UML sequence diagram.

![](images/vm_sequence_diagram.png)

Your actual task is to create or reuse three `Dockerfile`s and `docker-compose.yml` file. The `Dockerfile`s describe the setup of each single container and the `docker-compose.yml` describes how your application is started. That is, your application is developed and deployed as illustrated in the following UML deployment diagram.

![](images/vm_deployment.png)

Hints: 

  * In case you are using a webserver written in Go, the image `golang:jessie` is likely a good base for your container.
  * Check https://hub.docker.com for official images of various database engines.
  * In case you need an example webserver and database server, Helge could provide you those.

# Some more Info for the Group's Architects

As an interesting side note, you have just deployed a 2-tier and a 3-tier application. n-tier application layers are an architectural pattern. There are others, such as:

  * Client/Server
  * Peer-To-Peer
  * Repository
  * Model-View-Controller
  * Three-tier, Four-tier Architecture
  * Service-Oriented Architecture (SOA)
  * Pipes and Filters
  
See especially 6.3.5 and the entire chapter 6 in _"Object-Oriented Software Engineering Using UML, Patterns, and Java"_ Third Edition, Bernd Bruegge & Allen H. Dutoit.

Read more on Sequence Diagrams and other UML Diagrams in _"UML Distilled: A Brief Guide to the Standard Object Modeling Language"_ by Martin Fowler.

# Installing Docker

See the official documentation at https://docs.docker.com/engine/installation/ for guidance on installation. You will want to install the Docker Community Edition (CE).

## Windows

Follow the instructions under https://docs.docker.com/docker-for-windows/install/. Install the tool from the Stable Channel https://download.docker.com/win/stable/InstallDocker.msi and take special care of reading the section _"What to know before you install"_

## Linux 

On systems with APT there is likely a package woth the name `docker.io`, which can be installed via `sudo apt-get install docker.io`.

## MacOS

Follow the instructions under https://docs.docker.com/docker-for-mac/install/#what-to-know-before-you-install. Install the tool from the Stable Channel https://download.docker.com/mac/stable/Docker.dmg.

Anders provided the following guide on how to use networking with Docker containers from version 3 on.

# Linking containers in Docker 3

With version 3 of Docker, `--link` is deprecated.
Instead you should create bridging networks to connect containers.

If you create a container without specifying a network,
it will automatically be attached to the build in network called `bridge`.
In the `bridge` network all containers are given an unique IP,
but no hostname lookup is provided.

To get hostname lookup, you have to create your own network:

```bash
$ docker network create example-network
0e97d9478715b92da9ae4b2af8f219b70090f3a27e043186be27c8c9d467b694
```

You can see which networks exist with:

```bash
$ docker network ls
NETWORK ID          NAME                DRIVER              SCOPE
1820eab8890d        bridge              bridge              local
0e97d9478715        example-network     bridge              local
475e63c9fa31        host                host                local
24a15366ed0e        none                null                local
```

To connect a container to a custom network,
use the `--network` option.
Here two containers `example-1` and `example-2` are created:

```sh
$ docker run -itd --name=example-1 --network=example-network alpine:latest
e9ea1ca37d9bc248211d3eca2d623955a75084ea0e7d7e592c450e2b00a6f940
$ docker run -itd --name=example-2 --network=example-network alpine:latest
3b641b81bda3d9231e418aac5bf7a967c71ec48ea343d23e922a9117c3790c26
```
